In [2]:
import pandas as pd
from string import ascii_uppercase as alpha
import pickle
from bs4 import BeautifulSoup
import requests

In [25]:
all_tables=pd.read_html('https://web.archive.org/web/20221002180521/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [30]:
dict_tables={}
for letter,i in zip(alpha,range(11,67,7)):
    df=all_tables[i]
    df.rename(columns={df.columns[1]:'Team'},inplace=True)
    dict_tables[f"Cluster:{letter}"]=df

To csv Fixture..

In [31]:
df=pd.concat(dict_tables)
df.to_csv('Fixture_2022.csv',index=False)

In [23]:
with open('dict_tables.csv','wb') as output:
    pickle.dump(dict_tables,output)

Starting to filter the page

In [1]:
years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

making a request

In [41]:
web='https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response=requests.get(web)

Loading text with Beatifulsoup

In [ ]:
content=response.text
soup=BeautifulSoup(content,'lxml')

Filtering rows on matches

In [44]:
matches=soup.findAll('div',class_='footballbox')


Filtering Matches and adding to list for dataframe

In [55]:
home=[]
score=[]
away=[]
for match in matches:
    home.append(match.find('th',class_='fhome').get_text())
    score.append(match.find('th',class_='fscore').get_text())
    away.append(match.find('th',class_='faway').get_text())
#the elements have already been added to the lists

Creating the dictionary previusly for the Dataframe

In [66]:
dict_football={'home':home ,'score':score ,'Away':away}
df_football=pd.DataFrame(dict_football)
df_football['year']='2014'

To make the work easier, we are going to use a function, that does exactly the same as the previous thing..

In [5]:
def get_matches(year):
    web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response=requests.get(web)
    content=response.text
    soup=BeautifulSoup(content,'lxml')
    matches=soup.findAll('div',class_='footballbox')
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find('th',class_='fhome').get_text())
        score.append(match.find('th',class_='fscore').get_text())
        away.append(match.find('th',class_='faway').get_text())
    #the elements have already been added to the lists
    dict_football={'home':home ,'score':score ,'Away':away}
    df_football=pd.DataFrame(dict_football)
    df_football['year']=year
    return df_football

Let's try it..

In [71]:
#get_matches(1930)

we are going to save all the dataframes in a list

In [12]:
fifa=[get_matches(year)for year in years]
df_fifa=pd.concat(fifa,ignore_index=True)

To csv...

In [32]:
df_fifa.to_csv('Fifa_worldcup_historical_data1.csv',index=False)